# **🚀 LLaVA Model Deployment Notebook**

## 📖 **Overview**
Build a robust, production-ready pipeline that seamlessly combines powerful vision-language AI with scalable web deployment and e-commerce listing generation:

1. **Quantized LLaVA-Next / Mistral-7B Model** — Efficient multi-modal understanding  
2. **FastAPI Server** — Lightning-fast, production-grade API  
3. **Ngrok Tunneling** — Secure public endpoint for easy access  
4. **Multi-Image Product Analysis** — Detailed JSON output for rich insights  
5. **Mistral API Integration** — Automated generation of polished e-commerce listings  

## **🚀 Environment Setup**

Prepare your system with all necessary packages for lightning-fast model inference, quantization, and API hosting.  
We'll also clean up any old `transformers` versions to keep things smooth, then grab the latest cutting-edge release directly from GitHub! ⚡✨

In [ ]:
# System packages installation
!pip install -q accelerate transformers peft bitsandbytes trl flash-attn xformers
!pip install -q fastapi nest-asyncio pyngrok uvicorn python-multipart

# Clean up existing transformers installation
!pip uninstall -y transformers

# Install latest transformers from source
!pip install -q git+https://github.com/huggingface/transformers.git

!pip install torchvision

## 🔐 **Hugging Face Authentication**

Configure access to Hugging Face models:

- 🔑 **Token Setup**: Authenticate using your personal access token.
- 🌐 **Secure Access**: Enables downloading and using gated or private models from Hugging Face Hub.
- ⚡ **One-Time Configuration**: Required only once per session or environment.

In [ ]:
from huggingface_hub import HfFolder
import os

# Configure HF authentication
HfFolder.save_token("hf_YOUR_TOKEN_HERE")  # Replace with actual token
os.environ["HF_TOKEN"] = "hf_YOUR_TOKEN_HERE"

## ⚙️ **Model Initialization**

This section sets up the core models with performance-focused configurations:

- 🧩 **LLaVA-Next / Mistral-7B Quantized**: Loads the visual language model with quantized weights for reduced memory and faster inference.
- 🔧 **Optimized Pipeline**: Ensures efficient tokenizer, processor, and device assignment (e.g., GPU/CPU).
- 🚀 **Ready for Inference**: Models are configured to run in low-resource environments while maintaining output quality.

In [ ]:
import torch
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration, BitsAndBytesConfig
from PIL import Image

# Environment configuration
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64,garbage_collection_threshold:0.7"
os.environ["TRANSFORMERS_NO_FLASH_ATTENTION"] = "1"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Model initialization
processor = LlavaNextProcessor.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf",
    use_auth_token=True
)

model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf",
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    use_auth_token=True
)

# Compilation optimization
if hasattr(torch, "compile"):
    model = torch.compile(model, mode="reduce-overhead")

## 🚧 **Inference Pipeline**

This section implements the core logic for:

- 📸 **Image Preprocessing**: Resizes and prepares multiple input images for model inference.
- 🧠 **Prompt Construction**: Dynamically formats the input text prompt for visual grounding.
- 🗣️ **Text Generation**: Uses the quantized LLaVA model to generate structured output based on visual inputs.

Everything is optimized for fast batch processing and maximum token efficiency.

In [ ]:
def generate_fast(image_paths, prompt,
                  max_new_tokens=1024,
                  num_beams=1,
                  temperature=1.0,
                  img_size=160):
    # Build the full prompt by inserting one <image> token per image before the actual prompt
    image_token = processor.tokenizer.image_token or "<image>"
    placeholders = image_token * len(image_paths)
    full_prompt = f"{placeholders} {prompt}"

    # Load and preprocess all images: convert to RGB and resize to (img_size, img_size)
    images = [
        Image.open(p).convert("RGB").resize((img_size, img_size), Image.LANCZOS)
        for p in image_paths
    ]

    # Tokenize and batch the input prompt and images
    inputs = processor(images=images, text=full_prompt, return_tensors="pt").to(device)

    # Generate model output using the given generation parameters
    with torch.inference_mode():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            temperature=temperature,
            early_stopping=True,
            use_cache=True
        )

    # Decode and return the generated text output
    return processor.decode(out_ids[0], skip_special_tokens=True).strip()


## 🚀 **API Endpoint:** `/analyze-product`

This FastAPI endpoint handles the full flow for automated product listing generation:

- **Accepts**: Multiple uploaded product images via `POST`.
- **Processes**:
  - Saves images temporarily.
  - Sends them to the `generate_fast()` function (LLaVA model) to extract structured product details.
- **Chains to**: A Mistral backend (`/generate-listing`) to turn raw details into a formatted product listing.
- **Returns**: Final cleaned JSON object (`listing_section`) containing:
  - `Title`
  - `Category`
  - `Attributes`
  - `KeyFeatures`
  - `UseCase`
  - `Perfectly Accurate Detailed Description`

In [ ]:
import os
import requests
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Initialize FastAPI app
app = FastAPI()

# Define a temporary directory for image uploads
tmp_dir = "tmp_uploads"

@app.post("/analyze-product")
async def analyze_product(images: list[UploadFile] = File(...)):
    os.makedirs(tmp_dir, exist_ok=True)
    paths = []

    try:
        # Step 1: Save uploaded image files to disk
        for img_file in images:
            contents = await img_file.read()
            tmp_path = os.path.join(tmp_dir, f"{img_file.filename}")
            with open(tmp_path, 'wb') as f:
                f.write(contents)
            paths.append(tmp_path)

        # Step 2: Run LLaVA to extract visual information from images
        PROMPT = (
            "Extract everything you can see in the image(s) and output exactly one JSON object "
            "with keys: Title, Category, Attributes, KeyFeatures, UseCase, Perfectly Accurate Detailed Description. "
            "Only include details you can confirm visually."
            "Format the JSON properly with double quotes."
        )
        output = generate_fast(paths, PROMPT)

        # Log LLaVA output for debugging
        print("\n=== LLaVA RAW OUTPUT ===\n", output, "\n=========================\n")

        # Step 3: Send LLaVA output to Mistral API for further generation
        mistral_url = os.getenv("MISTRAL_API_URL", "MISTRAL-MODEL-NGROK-API-URL").rstrip("/") + "/generate-listing"
        try:
            resp = requests.post(mistral_url, json={"llava_output": output})
            mistral_output = resp.json()
        except Exception as e:
            mistral_output = {"error": f"Failed to call Mistral: {e}"}

        # Log Mistral output for debugging
        print("\n=== MISTRAL LISTING OUTPUT ===\n", mistral_output, "\n=============================\n")

        # Step 4: Extract only the final JSON listing section from Mistral's output
        raw_text = mistral_output.get('raw_mistral_output', "")
        last_title_idx = raw_text.rfind('"title"')

        if last_title_idx != -1:
            start_idx = raw_text.rfind('{', 0, last_title_idx)
            end_idx = raw_text.rfind('}')
            if start_idx != -1 and end_idx != -1 and end_idx > start_idx:
                listing_section = raw_text[start_idx:end_idx + 1]
            else:
                listing_section = raw_text
        else:
            listing_section = raw_text

        print("=== LISTING SECTION ===", listing_section, "========================")

        # Step 5: Return the structured listing section as JSON
        return JSONResponse({
            "listing_section": listing_section
        })

    finally:
        # Step 6: Clean up uploaded image files
        for p in paths:
            if os.path.exists(p):
                os.remove(p)


# Start FastAPI server with ngrok tunneling
NGROK_TOKEN = "YOUR-NGROK-TOKEN-HERE"  # Replace with your own
ngrok.set_auth_token(NGROK_TOKEN)
ngrok_tunnel = ngrok.connect(8000)
print(f'🌐 LLaVA API URL: {ngrok_tunnel.public_url}')

# Enable asyncio compatibility
nest_asyncio.apply()

# Run FastAPI app using Uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)
